In [3]:
from pathlib import Path
from osgeo import gdal

gdal.UseExceptions()

In [42]:
input_rgb = Path("/Volumes/Sammy/terra-luma/stac-input/EvelynDowns/SANSSTP005/20241208_SANSSTP005_m3m_60mAGL_ortho_rgb.tif")
input_ms = Path("/Volumes/Sammy/terra-luma/stac-input/EvelynDowns/SANSSTP005/20241208_SANSSTP005_m3m_60mAGL_ortho_ms.tif")

In [50]:
def convert_ortho(input: Path):
    # Assert is a .tif file
    assert(input.is_file())
    assert(input.suffix == '.tif')

    # Create tmp and output file paths
    tmp_file = input.with_suffix('.tmp.tif')
    output = input.with_suffix('.cog.tif')

    # Read input file info
    input_info = gdal.alg.raster.info(input=input).Output()

    # Assert is a GeoTiff file
    assert(input_info['driverShortName'] == 'GTiff')

    # Assert is 3 or 4 bands
    num_bands = len(input_info['bands'])
    assert(num_bands == 3 or num_bands == 4)

    # Assert byte type for 3 band RGB and float32 for 4 band MS
    if num_bands == 3:
        assert(input_info['bands'][0]['type'] == 'Byte')
    elif num_bands == 4:
        assert(input_info['bands'][0]['type'] == 'Float32')

    # Default nodata 255 for 3 band RGB and -32767.0 for 4 band MS
    if num_bands == 3:
        nodata = 255
    elif num_bands == 4:
        nodata = -32767.0

    # Check if there already is a nodata value and use that instead
    if 'noDataValue' in input_info['bands'][0]:
        nodata = input_info['bands'][0]['noDataValue']

    # Copy src to a tmp file not to change anything
    # In the future, we may be copying from a remote src
    gdal.alg.dataset.copy(source=input, destination=tmp_file, overwrite=True)

    # Edit temp file to:
    #   - Add nodata values
    #   - Compute statistics
    gdal.alg.raster.edit(dataset=tmp_file, nodata=nodata, stats="YES")

    # Convert to COG with
    #   - ZSTD compression (lossless)
    #   - Predictor = YES (uses appropriate level for data type)
    #   - Level = 9 (amount of compression, default for ZSTD)
    gdal.alg.raster.convert(
        input=tmp_file,
        output=output,
        output_format="COG",
        creation_option={
            "COMPRESS": "ZSTD", 
            "LEVEL": 9, 
            "PREDICTOR": "YES"
        },
        overwrite=True
    )

    # Delete temp file
    gdal.alg.dataset.delete(filename=tmp_file)
    